In [3]:
import re
import pandas as pd
from forex_python.converter import CurrencyRates

In [5]:
raw = pd.read_csv('final.csv')
raw = pd.DataFrame(raw)
data = raw

In [81]:
set(data.budget.apply(lambda x: x[0]))
data[data.budget.str.startswith('\xc2')]['budget']

196      £15,000,000
248       £1,900,000
252         £612,650
256       £4,000,000
307       £1,500,000
382       £4,000,000
407      £25,000,000
424       £3,500,000
521       £1,000,000
670      £20,000,000
689     £150,000,000
737       £4,000,000
743       £4,500,000
747       £3,000,000
767       £6,000,000
954       £8,000,000
1154        £229,575
1160        £960,000
1165     £26,000,000
1270      £2,000,000
1281     £11,000,000
1416     £10,000,000
1422      £4,900,000
1507      £2,200,000
1521      £4,000,000
1538      £2,500,000
1540     £13,000,000
1560      £1,700,000
1577      £2,500,000
1861         £14,000
1891        £120,000
1925      £5,000,000
2072        £500,000
2129      £1,500,000
2149      £1,000,000
2166      £3,000,000
2174      £2,800,000
2213      £7,000,000
2375      £4,000,000
2527      £3,500,159
2772      £6,400,000
2782        £500,000
2788      £2,000,000
2835     £10,000,000
2849      £1,000,000
2865      £7,000,000
2867        £450,000
2881     £13,

In [70]:
x=re.sub('\d',"",data.budget[28])
x=re.sub(',',"", x)
x=re.sub('\xc2\xa0',"",x)
cur = symbols.get(x,x)
cur

'EUR'

## 

In [58]:
currencies = []
symbols = {
        "\xe2\x82\xac": 'EUR',
        "\xc2\xa3": 'GBP'}

for i in data.budget:
    if i[0] != '$':
        x = re.sub('\d',"",i)
        x = re.sub(',',"", x)
        x = re.sub('\xc2\xa0',"",x)
        cur = symbols.get(x,x)
        currencies.append(cur)

        
currencies = list(set(currencies))
currencies

['FRF',
 'AUD',
 'NOK',
 'KRW',
 'INR',
 'CNY',
 'DEM',
 'GBP',
 'NZD',
 'PLN',
 'EUR',
 'DKK',
 'THB',
 'HKD',
 'CAD']

In [42]:
badCurrencies = ['DEM','FRF']

In [54]:
c = CurrencyRates()

fx = {'USD': 1,
     'DEM': 0,
     'FRF': 0,
     'ITL': 0,
     'SGD': 0}
fx['JPY'] = c.get_rate(i, 'USD')

for i in currencies:
    if i not in badCurrencies:
        fx[i] = c.get_rate(i, 'USD')

In [44]:
def convertCurrency(stringNum):
    symbols = {
        "\xe2\x82\xac": 'EUR',
        "\xc2\xa3": 'GBP'
    }
    
    num = int("".join(re.findall(r'\d+',stringNum)))
    
    if stringNum[0] != '$':
        x = re.sub('\d',"",stringNum)
        x = re.sub(',',"", x)
        x = re.sub('\xc2\xa0',"",x)
        cur = symbols.get(x,x)
        cur = fx[cur]
        
        return num * cur
    return num

In [64]:
data.budget = data.budget.apply(convertCurrency)

In [65]:
data.gross = data.gross.apply(convertCurrency)

In [66]:
data.open_week = data.open_week.map(lambda x: x.split('\n')[0].strip())

In [67]:
data.open_week = data.open_week.apply(convertCurrency)

In [71]:
data['rating'] = data['date_c'].map(lambda x: x.split(',')[0].strip())

In [74]:
data.head()

,Unnamed: 0,gross,date_c,name,rate,budget,Director,length,actors,open_week,year,genre,rating,audience,First_Week
0,0,77264926,"PG,2012-12-25",Parental Guidance,"6.1 based on 21,923 user ratings",25000000.0,Andy Fickman,105M,"Billy Crystal,Bette Midler,Marisa Tomei",1.455405e+07,2012,"Comedy,Family",PG,"21,923","$14,554,053"
1,1,69148997,"PG-13,2004-06-23",White Chicks,"5.5 based on 104,311 user ratings",37000000.0,Keenen Ivory Wayans,109M,"Marlon Wayans,Shawn Wayans,Busy Philipps",1.967675e+07,2004,"Comedy,Crime",PG-13,"104,311","$19,676,748"
2,2,107930000,"R,1994-10-14",Pulp Fiction,"8.9 based on 1,412,769 user ratings",8000000.0,Quentin Tarantino,154M,"John Travolta,Uma Thurman,Samuel L. Jackson",5.490319e+06,1994,"Crime,Drama",R,"1,412,769","£4,243,233"
3,3,71277420,"R,2001-07-04",Scary Movie 2,"5.3 based on 125,077 user ratings",45000000.0,Keenen Ivory Wayans,83M,"Anna Faris,Marlon Wayans,Antony Acker",2.050336e+07,2001,Comedy,R,"125,077","$20,503,356"
4,4,79235468,"PG-13,2011-03-18",Limitless,"7.4 based on 442,994 user ratings",27000000.0,Neil Burger,105M,"Bradley Cooper,Anna Friel,Abbie Cornish",1.890730e+07,2011,"Mystery,Sci-Fi,Thriller",PG-13,"442,994","$18,907,302"


In [75]:
z = "ABCS"
z[0:1]

'A'

In [77]:
notPG = data.rating.map(lambda x: x[0:2] != 'PG')

In [80]:
data[notPG]

,Unnamed: 0,gross,date_c,name,rate,budget,Director,length,actors,open_week,year,genre,rating,audience,First_Week
2,2,107930000,"R,1994-10-14",Pulp Fiction,"8.9 based on 1,412,769 user ratings",8000000.0,Quentin Tarantino,154M,"John Travolta,Uma Thurman,Samuel L. Jackson",5.490319e+06,1994,"Crime,Drama",R,"1,412,769","£4,243,233"
3,3,71277420,"R,2001-07-04",Scary Movie 2,"5.3 based on 125,077 user ratings",45000000.0,Keenen Ivory Wayans,83M,"Anna Faris,Marlon Wayans,Antony Acker",2.050336e+07,2001,Comedy,R,"125,077","$20,503,356"
6,6,37371385,"R,2011-03-18",Paul,"7.0 based on 201,052 user ratings",40000000.0,Greg Mottola,104M,"Simon Pegg,Nick Frost,Seth Rogen",7.138603e+06,2011,"Adventure,Comedy,Sci-Fi",R,"201,052","£5,517,121"
7,9,22858926,"R,1999-12-03",Being John Malkovich,"7.8 based on 264,768 user ratings",13000000.0,Spike Jonze,112M,"John Cusack,Cameron Diaz,Catherine Keener",6.377210e+05,1999,"Comedy,Drama,Fantasy",R,"264,768","$637,721"
8,11,42575718,"R,2011-08-12",Final Destination 5,"5.9 based on 87,764 user ratings",40000000.0,Steven Quale,92M,"Nicholas D'Agosto,Emma Bell,Arlen Escarpeta",1.803140e+07,2011,"Horror,Thriller",R,"87,764","$18,031,396"
11,14,101334374,"R,1997-12-12",Scream 2,"6.1 based on 130,420 user ratings",24000000.0,Wes Craven,120M,"Neve Campbell,Courteney Cox,David Arquette",3.300000e+07,1997,"Horror,Mystery",R,"130,420","$33,000,000"
12,15,60491560,"R,1997-10-03",Kiss the Girls,"6.6 based on 53,121 user ratings",27000000.0,Gary Fleder,115M,"Morgan Freeman,Ashley Judd,Cary Elwes",1.321517e+07,1997,"Crime,Drama,Mystery",R,"53,121","$13,215,167"
14,17,24809547,"R,2011-08-26",Our Idiot Brother,"6.4 based on 76,949 user ratings",5000000.0,Jesse Peretz,90M,"Paul Rudd,Elizabeth Banks,Zooey Deschanel",7.011631e+06,2011,"Comedy,Drama",R,"76,949","$7,011,631"
15,19,16816,"R,2011-10-21",Trespass,"5.3 based on 35,781 user ratings",35000000.0,Joel Schumacher,91M,"Nicolas Cage,Nicole Kidman,Cam Gigandet",1.681600e+04,2011,"Crime,Thriller",R,"35,781","$16,816"
17,23,1729969,"R,2011-11-10",Take Shelter,"7.4 based on 73,903 user ratings",5000000.0,Jeff Nichols,120M,"Michael Shannon,Jessica Chastain,Shea Whigham",5.204100e+04,2011,"Drama,Thriller",R,"73,903","$52,041"
